In [1]:
import tensorflow as tf
import numpy as np
import warnings
import os.path as osp
import time

from scipy.stats import entropy
from general_tools.simpletons import iterate_in_chunks

from tf_lab.external.Chamfer_EMD_losses.tf_nndistance import nn_distance
from tf_lab.external.Chamfer_EMD_losses.tf_approxmatch import approx_match, match_cost

import time

In [ ]:
l_h = np.load('l_emd_hist_test.npz')
l_h = l_h[l_h.keys()[0]]


r_h = np.array(all_d['test'])


bins = np.linspace(-0.000001, np.max([r_h, l_h]), 100)               

plt.hist(r_h, bins, alpha=0.5, label='raw')
plt.hist(l_h, bins, alpha=0.5, label='l_emd')
plt.legend(loc='upper right')

